In [1]:
'''
Worked on by: Meena Hari and Tarini Singh.

We perform data preprocessing using KNearestNeighbors.
66 new features are generated.

Trained a 1 layer ANN with transformed, higher dimensional 
dataset (each input consists of the raw board representaion 
(list of integers from 1 - 16) plus 66 newly generated features).

In prog.

'''
from numba import njit, jit
import random
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, Flatten, Input
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.models import load_model
import keras.losses

from constants import *
import heuristic as h
import io_help as io
import neural_net as nn
import solver as s

NEIGHBORS = 151

def load_data(file_name):
    """
    This function reads in training data from a file and returns 
    the boards in X and their labels in Y as a tuple. 
    """
    file = open(file_name, "r")
    X = []
    Y = []
    

    for string in file: 
        (board, dist) = io.string_to_board_and_dist(string)
        X_temp = np.concatenate((board.reshape(16)), axis=None)
        X.append(X_temp)
        Y.append(dist)
        
    file.close()
    X_train = np.asarray(X)
    Y_train = np.asarray(Y)
    return(X_train, Y_train)

Using TensorFlow backend.


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/meena/miniconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
# Load dataset. 
# X: board inputs, Y: true output.
(X_train,Y_train) = load_data('Uncombined Data Files/meena_5_19_2020_93844.txt')
print(X_train.shape)

(93844, 16)


In [25]:
# Generates additional features.
# X: the input data file.
# X_train: the original training data file (not transformed).


#def gen_features (X, X_train, knn_model):

#def gen_features(X, neighs, X_train, knn_model):
@njit
def gen_features(X, neighs, X_train):
    data_arr = np.zeros((len(X), 66), dtype=float64)
    man_ham_2D = np.zeros((len(X), 66), dtype=float64)
    one_hot_2D = np.zeros((len(X), 288), dtype=float64)
    #pred = knn_model.kneighbors(X)
    
    #for i in tqdm(range(len(X))):
    for i in range(X.shape[0]):
        row = X[i]
        # Grabs the rows in X corresponding to 50 nearest neighbors of X[i].
        # pred[1][i] contains a list of the indices of the 50 nearest neighbors.
        data = X_train[neighs[i]]
        # Divide X[i] by each of its neighbors. div should be a 
        # 151 x 16 matrix, i.e. div[j] = X[i] / X[j].
        div = (row / data)
        # Subtract X[i] by each of its neighbors. diff should be a 
        # 151 x 16 dimension matrix.
        diff = (row - data)
        
        
        # concat is a 151 x 32 matrix.
        concat = np.zeros((NEIGHBORS, 32))
        concat[:, :16] = div
        concat[:, 16:] = diff
        #concat = np.concatenate([div, diff], axis = 1)
        
        # means is a 151 x 32 matrix.
        # std is a 151 x 32 matrix.
        means, stds = np.nanmean(concat, axis = 0), np.nanstd(concat, axis = 0)
        # Populate data_arr with newly generated features.
        data_arr[i, :means.shape[0]] = means
        data_arr[i, means.shape[0]:means.shape[0] + stds.shape[0]] = stds
        data_arr[i, -1] = np.nanmean(pred[0][i])
        data_arr[i, -2] = np.nanstd(pred[0][i])
        
        
        # Hamming distances
        """
        computes the hamming metric from the board to the solved state and returns 
        it
        trash is merely to get to conform to standard metric implementation
        https://en.wikipedia.org/wiki/Hamming_distance
        """
        ham_dist = 0
        board = row.reshape(4,4)
        for i in range (0, SIZE):
            for j in range (0, SIZE):
                # check of (2 * (SIZE - 1)) ensures that do not check 
                val = board[i,j]
                if i + j != (2 * (SIZE - 1)) and (i,j) != (((val-1) // SIZE), ((val-1) % SIZE)):
                    ham_dist += 1
        
        
        # Manhattan Distance
        """
        computes the manhattan metric from the board to the solved stateand returns 
        it
        trash is merely to get to conform to standard metric implementation
        https://en.wikipedia.org/wiki/Taxicab_geometry
        """
        man_dist = 0
        for i in range (0, SIZE):
            for j in range (0, SIZE):
                val = board[i,j]
                (i_right, j_right) = (((val-1) // SIZE), ((val-1) % SIZE))
                if board[i,j] != SIZE ** 2:
                    man_dist += abs(i - i_right) + abs(j - j_right)
        
        man_ham_2D[i,0] = man_dist
        man_ham_2D[i,1] = ham_dist
        #one_hot_2D[i] = nn.get_rep_2(row.reshape(4,4))
        
    # Concatenate generated features to the original dataset.
    #return np.concatenate([one_hot_2D, man_ham_2D, data_arr], axis=1)
    return (man_ham_2D, data_arr)
    #return np.concatenate([man_ham_2D, data_arr], axis=1)

In [5]:
knn_model = NearestNeighbors(n_neighbors=NEIGHBORS, n_jobs = -1).fit(X_train,Y_train)


In [6]:
pred = knn_model.kneighbors(X_train)

In [26]:
#X_train_2 = gen_features(X_train, pred[1], X_train)
(man_ham_2D, data_arr) = gen_features(X_train, pred[1], X_train)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<function nanmean at 0x10dc43ea0>) with argument(s) of type(s): (array(float64, 2d, C), axis=Literal[int](0))
 * parameterized
In definition 0:
    TypeError: np_nanmean() got an unexpected keyword argument 'axis'
    raised from /Users/meena/miniconda3/lib/python3.7/site-packages/numba/typing/templates.py:539
In definition 1:
    TypeError: np_nanmean() got an unexpected keyword argument 'axis'
    raised from /Users/meena/miniconda3/lib/python3.7/site-packages/numba/typing/templates.py:539
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<function nanmean at 0x10dc43ea0>)
[2] During: typing of call at <ipython-input-25-cdaf72806533> (38)


File "<ipython-input-25-cdaf72806533>", line 38:
def gen_features(X, neighs, X_train):
    <source elided>
        # std is a 151 x 32 matrix.
        mea

In [ ]:
X_train_2 = np.concatenate([man_ham_2D, data_arr], axis=1)

In [8]:
X_train_2

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.12507262, 1.06638986,
        8.61935475],
       [0.        , 0.        , 0.        , ..., 0.13267004, 1.14509462,
        8.38873516],
       [0.        , 0.        , 0.        , ..., 0.1328004 , 1.10117625,
        8.63494327]])

In [9]:
def shift_mse(y_true, y_pred):
    """custom loss functions"""
    loss = (1 + 6/(1 + K.exp(-(y_pred - y_true)))) * K.square(y_pred - y_true)
    loss = K.mean(loss, axis = 1)
    return loss


def exp_loss_2(y_true, y_pred):
    """
    Custom loss function. 
    """
    loss = K.exp((y_pred - y_true))
    loss = loss + K.square(y_pred - y_true)
    loss = K.mean(loss, axis = 1)

    return loss
    
keras.losses.shift_mse = shift_mse
keras.losses.exp_loss_2 = exp_loss_2

In [10]:
def luka_model (X, Y):
    # Build Model
    model = Sequential()

    # Input Layer
    i = Input(shape = (16*2*2+2+(256 + 32 + 2),))
    x_1 = Dense(16*2*2+2+(256 + 32 + 2), activation='relu')(i)
    x_2 = Dropout(0.1)(x_1)
    x_3 = Dense(356, activation='relu')(x_2)
    x_4 = Dropout(0.1)(x_3)
    x_5 = Dense(17, activation='relu')(x_4)
    o = Dense(1, activation='linear')(x_1)
    model = Model(i,o)

    # Define the optimizer and loss function
    model.compile(optimizer='adam', loss=exp_loss_2, metrics=['accuracy'])

    # You can also define a custom loss function
    # model.compile(optimizer='adam', loss=custom_loss)

    # Train 
    model.fit(X, Y, epochs=15)

    return model

In [ ]:
'''
# Build Model
model = Sequential()

# Input Layer
model.add(Dense(units=(16*2*2+2+16), input_dim=(16*2*2+2+16), activation='relu'))
model.add(Dropout(0.1))

# Hidden Layers
model.add(Dense(units=66+16, activation='relu'))

# Output Layer
model.add(Dense(units=1, activation='linear'))

# Define the optimizer and loss function
#model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.compile(optimizer='adam', loss=shift_mse, metrics=['accuracy'])

# You can also define a custom loss function
# model.compile(optimizer='adam', loss=custom_loss)

# Train 
model.fit(X_train_2, Y_train, epochs=20)

# Test
#score = model.evaluate(X_test, Y_test)

#print(score)
'''

In [11]:
# Load test dataset. 
# X_test: board inputs, Y_test: true output.
(X_test,Y_test) = load_data('Uncombined Data Files/Yasmin_5_19_10048.txt')

# Transform X_test to higher dimension.
X_test_2 = gen_features (X_test, X_train, knn_model)

100%|██████████| 10047/10047 [00:05<00:00, 1705.58it/s]


In [13]:
dist_over_i = []
misclass_i = 0
dist_under_i = []
dist_over_man_i = []
dist_under_man_i = []

dist_over_r = []
misclass_r = 0
dist_under_r = []

model = luka_model(X_test_2, Y_test)

for i in tqdm(range(len(X_test))):
    nn_heur_i = int(model.predict(X_test_2[i:(i+1),:]))
    nn_heur_r = np.around(model.predict(X_test_2[i:(i+1),:]))
    man_heur = h.manhattan(X_test[i].reshape(4,4), model)
    y = Y_test[i]
    
    ### TRUNCATE ###
    if (nn_heur_i > y):
        dist_over_i.append(nn_heur_i - y)
    
    if (nn_heur_i <= y):
        dist_under_i.append(y - nn_heur_i)
    
    if (nn_heur_i != y):
        misclass_i += 1
    
    if (nn_heur_i > man_heur):
        dist_over_man_i.append(nn_heur_i - man_heur)
        
    if (nn_heur_i < man_heur):
        dist_under_man_i.append(man_heur - nn_heur_i)
    
        
    ##### ROUND ##### 
    if (nn_heur_r > y):
        dist_over_r.append(nn_heur_r - y)
    
    if (nn_heur_r <= y):
        dist_under_r.append(y - nn_heur_r)
    
    if (nn_heur_r != y):
        misclass_r += 1
    
avg_dist_over_i = np.mean(np.asarray(dist_over_i))
avg_dist_under_i = np.mean(np.asarray(dist_under_i))
out_sample_error_i = misclass_i / len(X_test)
avg_dist_over_man_i = np.mean(np.asarray(dist_over_man_i))
avg_dist_under_man_i = np.mean(np.asarray(dist_under_man_i))

avg_dist_over_r = np.mean(np.asarray(dist_over_r))
avg_dist_under_r = np.mean(np.asarray(dist_under_r))
out_sample_error_r = misclass_r / len(X_test)
 
print("------ TRUCATION: ------")
print("Avg distance overestimated: ", avg_dist_over_i)
print("Avg distance underestimated: ", avg_dist_under_i)
print("E_admiss: ", len(dist_over_i)/len(X_test))
print("E_out: ", out_sample_error_i)
print("Avg distance over Manhattan: ", avg_dist_over_man_i)
print("Avg distance under Manhattan: ", avg_dist_under_man_i)
print("Percent over Manhattan: ", len(dist_over_man_i)/len(X_test))
print("Percent under Manhattan: ", len(dist_under_man_i)/len(X_test))
print("\n")
print("------ ROUNDED: ------")
print("Avg distance overestimated: ", avg_dist_over_r)
print("Avg distance underestimated: ", avg_dist_under_r)
print("E_admiss: ", len(dist_over_r)/len(X_test))
print("E_out: ", out_sample_error_r)

Epoch 1/15
10047/10047 [==============================] - 1s 71us/step - loss: 20.8628 - accuracy: 0.2501
Epoch 2/15
10047/10047 [==============================] - 0s 47us/step - loss: 6.9038 - accuracy: 0.3425
Epoch 3/15
10047/10047 [==============================] - 0s 49us/step - loss: 5.6089 - accuracy: 0.3629
Epoch 4/15
10047/10047 [==============================] - 0s 50us/step - loss: 4.6231 - accuracy: 0.3744
Epoch 5/15
10047/10047 [==============================] - 1s 66us/step - loss: 3.8167 - accuracy: 0.3957
Epoch 6/15
10047/10047 [==============================] - 1s 55us/step - loss: 3.1482 - accuracy: 0.4097
Epoch 7/15
10047/10047 [==============================] - 0s 50us/step - loss: 2.6632 - accuracy: 0.4303
Epoch 8/15
10047/10047 [==============================] - 0s 50us/step - loss: 2.2559 - accuracy: 0.4809
Epoch 9/15
10047/10047 [==============================] - 0s 49us/step - loss: 1.9696 - accuracy: 0.4828
Epoch 10/15
10047/10047 [=============================

100%|██████████| 10047/10047 [00:08<00:00, 1195.01it/s]

------ TRUCATION: ------
Avg distance overestimated:  1.0057803468208093
Avg distance underestimated:  0.595705894267774
E_admiss:  0.017219070369264455
E_out:  0.5658405494177366
Avg distance over Manhattan:  4.451914270012823
Avg distance under Manhattan:  1.0125414976023608
Percent over Manhattan:  0.5433462725191599
Percent under Manhattan:  0.269831790584254


------ ROUNDED: ------
Avg distance overestimated:  1.0059022
Avg distance underestimated:  0.17210247
E_admiss:  0.11804518761819449
E_out:  0.25868418433363194


In [14]:
model.save("one_HAWT_356_17_nn.txt")

In [22]:
'''
# DON'T HAVE TO RUN THIS CELL AGAIN, only to transform Yasmin_5_16_40360.txt.

(X,Y) = load_data('Uncombined Data Files/Yasmin_5_16_40360.txt')
knn_model_all = NearestNeighbors(n_neighbors=50, n_jobs = -1).fit(X,Y)
X_2 = gen_features(X, X, knn_model_all)
X_Y = np.column_stack((X_2, Y))
np.savetxt("Yasmin_40360_50knn_Trans.csv", X_Y, delimiter=',')
'''

'\n# DON\'T HAVE TO RUN THIS CELL AGAIN, only to transform Yasmin_5_16_40360.txt.\n\n(X,Y) = load_data(\'Uncombined Data Files/Yasmin_5_16_40360.txt\')\nknn_model_all = NearestNeighbors(n_neighbors=50, n_jobs = -1).fit(X,Y)\nX_2 = gen_features(X, X, knn_model_all)\nX_Y = np.column_stack((X_2, Y))\nnp.savetxt("Yasmin_40360_50knn_Trans.csv", X_Y, delimiter=\',\')\n'

In [15]:
def string_to_test_info(string):
    """
    given a string containing the standard form of test info, returns tuple of 
    board, number of states to solution, time, and lenght of solution
    """
    split = string.split("!")
    board = io.string_to_board(split[0])
    n_states = int(split[1])
    time = float(split[2])
    sol_len = int(split[3])
    return (board, n_states, time, sol_len)


def load_boards(filename):
    """
    given name of file containing test boards, loads all test boards
    """
    file = open(filename, "r")

    boards = []
    n_states = []
    times = []
    dists = []

    for line in file:
        (board, c_states, c_time, sol_len) = string_to_test_info(line)
        boards.append(board)
        n_states.append(c_states)
        times.append(c_time)
        dists.append(sol_len)

    return (boards, n_states, times, dists)

def run_testing(data_file, model, h_func):
    """
    given a data_file containing testing data, a model, and heuristic function
    for said model, computes average number of states to solution, number to 
    times solution length is non-optimal, and average estimates of solution 
    lengths
    """
    (boards, n_states, times, dists) = load_boards(data_file)

    cust_states = []
    cust_wrong = 0
    cust_distance = []

    for i in tqdm(range(len(boards))):
        #(c_states, c_time, sol_path) = s.solve(boards[i], h_func, model)
        (c_states, c_time, sol_path) = s.solve(boards[i], h_func, model)
        cust_states.append(c_states)
        sol_len = len(sol_path) - 1
        if not (sol_len == dists[i]):
            cust_wrong += 1
        cust_distance.append(sol_len)

    print("average number of states explored to find solution:")
    print("\tfor learned model: " + str(np.mean(cust_states)))
    print("\tfor manhattan distance: " + str(np.mean(n_states)))
    print("----------------------------------------------------")
    print("solution was non-optimal " + str(cust_wrong / NUM_TEST_BOARDS * 100) + "% of the time")
    print("----------------------------------------------------")
    print("average length of solution path was:")
    print("\tfor learned model: " + str(np.mean(cust_distance)))
    print("\tfor manhattan distance: " + str(np.mean(dists)))
'''
def heur_boi(board, model):
    """
    This function takes in a board and a trained NN model and returns
    the heuristic the model predicts.
    """
    return 0
    #[[pred]] = model.predict(board)
    #return round(pred)
'''

'\ndef heur_boi(board, model):\n    """\n    This function takes in a board and a trained NN model and returns\n    the heuristic the model predicts.\n    """\n    return 0\n    #[[pred]] = model.predict(board)\n    #return round(pred)\n'

In [16]:
dic = {}
def heur_boi(board, model):
    b1 = board.reshape(16)
    man = np.array(h.manhattan(b1.reshape(4,4), None))
    ham = np.array(h.hamming(b1.reshape(4,4), None))
    
    b1_str = np.array_str(b1)
    #b2 = np.concatenate((b1, man, ham), axis=None)
    #b2_str = np.array_str(b2)
    
    if b1_str in dic:
        return dic.get(b1_str)
    else:
        # transform board
        X_2 = gen_features(np.asarray([b1]), X_train, knn_model)
        [[pred]] = model.predict(X_2[0].reshape(1,-1))
        dic[b1_str] = int(pred)
        return dic[b1_str]

In [ ]:
run_testing('baby_test.txt', model, heur_boi)

 67%|██████▋   | 10/15 [12:08<04:41, 56.25s/it]

In [ ]:
model.predict(np.zeros((1,82)))

In [ ]:
(np.arange(82).T).shape

In [ ]:
np.arange(82)

In [ ]:
np.asarray([np.arange(82)])

In [ ]:
t2 = gen_features(np.asarray([np.arange(16)]), X_train, knn_model)

In [ ]:
t2

In [ ]:
t3 = gen_features(np.asarray([np.arange(16)]), X_train, knn_model)
[[pred]] = model.predict(t3[0].reshape(1,-1))
print(round(pred))

In [ ]:
np.array([2,6,3,4,12,5,10,15,1,14,16,8,13,7,9,11])

In [ ]:
t3 = gen_features(np.asarray([np.array([5,7,16,2,6,14,12,1,9,3,11,15,13,10,8,4])]), X_train, knn_model)
[[pred]] = model.predict(t3[0].reshape(1,-1))
print(round(pred))

In [ ]:
nn.calc_displacements(np.array([5,7,16,2,6,14,12,1,9,3,11,15,13,10,8,4]).reshape(4,4)).shape

In [ ]:
t4 = np.zeros()

In [65]:
def dummy(arr):
    for i in range(100000):
        arr = arr / 16
    return arr

%time dummy(np.arange(16))

CPU times: user 137 ms, sys: 2.61 ms, total: 140 ms
Wall time: 139 ms


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [66]:
@jit
def dummy2(arr):
    for i in range(100000):
        arr = arr / 16.0
    return arr

In [67]:
%time dummy2(np.arange(16.0))

CPU times: user 136 ms, sys: 2.51 ms, total: 138 ms
Wall time: 137 ms


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [68]:
%time dummy2(np.arange(16.0))

CPU times: user 14.8 ms, sys: 308 µs, total: 15.1 ms
Wall time: 15 ms


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [55]:
def monte_carlo(n):
    acc = 0
    for i in range(n):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / n

In [62]:
%time dummy(10000)

CPU times: user 4.84 ms, sys: 387 µs, total: 5.23 ms
Wall time: 4.92 ms


3.1792

In [63]:
mc2 = jit()(monte_carlo)

%time mc2(10000)

CPU times: user 89.6 ms, sys: 2.18 ms, total: 91.8 ms
Wall time: 91.2 ms


3.1276

In [64]:
%time mc2(10000)

CPU times: user 183 µs, sys: 1 µs, total: 184 µs
Wall time: 186 µs


3.1252

CPU times: user 1.76 ms, sys: 2 µs, total: 1.77 ms
Wall time: 1.77 ms


3.13416

In [76]:
np.zeros(16)

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [87]:
float64(3)

3.0